In [23]:
from scqbf.scqbf_evaluator import *
from scqbf.scqbf_solution import *
from scqbf.scqbf_grasp import *
from scqbf.scqbf_instance import *

import pandas as pd
import numpy as np
import pickle

In [24]:
instance_paths = [(f"instances/gen{i}/instance{j}.txt", i, j) for i in range(1, 4) for j in range(1, 6)]

In [25]:
alpha_1 = 0.3   # More greedy
alpha_2 = 0.8   # More random

In [ ]:
def run_experiment(config: dict) -> pd.DataFrame:
    results = []
    exp_num = 1

    for instance_path, gen, inst in instance_paths:
        instance = read_max_sc_qbf_instance(instance_path)
        print(f"{exp_num}: {inst}th instance of generation strategy {gen}. Path: {instance_path}")
        exp_num += 1
        
        grasp = ScQbfGrasp(instance, config=config, max_iterations=None, time_limit_secs=600, patience=1)
        best_solution = grasp.solve()
        
        if grasp.solve_time >= 600:
            stop_reason = "time_limit"
        else:
            stop_reason = "patience_exceeded" 
        
        evaluator = ScQbfEvaluator(instance)
        
        results.append({
            'gen': gen,
            'inst': inst,
            'n': instance.n,
            'stop_reason': stop_reason,
            'best_objective': evaluator.evaluate_objfun(best_solution),
            'coverage': evaluator.evaluate_coverage(best_solution),
            'time_taken': grasp.solve_time
        })
        
        last_result = results[-1]
        print(f"\tBest objective value: {last_result['best_objective']:.4f}")
        print(f"Selected elements: {best_solution.elements}")
        print(f"\tCoverage: {last_result['coverage']:.2%}")
        print(f"\tTime taken (secs): {last_result['time_taken']:.4f} s")
        print(f"\tStop reason: {last_result['stop_reason']}")
        print()
    
    df = pd.DataFrame(results)
    return df

## Configuração 1 - PADRÃO:
GRASP com parâmetro α1 , first-improving e heurística construtiva padrão.

In [27]:
config_1 = {
    "construction_method": "traditional",
    "construction_args": [alpha_1],
    "local_search_method": "first_improve",
}
results_df = run_experiment(config_1)

with open("pickles/results_config_1.pkl", "wb") as f:
    pickle.dump(results_df, f)

1: 1th instance of generation strategy 1. Path: instances/gen1/instance1.txt
Patience exhausted, no improvement to the objective solution in 1 iterations, stopping GRASP.
	Best objective value: 4590.4300
	Coverage: 100.00%
	Time taken (secs): 0.2638 s
	Stop reason: patience_exceeded

2: 2th instance of generation strategy 1. Path: instances/gen1/instance2.txt
Patience exhausted, no improvement to the objective solution in 1 iterations, stopping GRASP.
	Best objective value: 4668.0700
	Coverage: 100.00%
	Time taken (secs): 0.9374 s
	Stop reason: patience_exceeded

3: 3th instance of generation strategy 1. Path: instances/gen1/instance3.txt
Patience exhausted, no improvement to the objective solution in 1 iterations, stopping GRASP.
	Best objective value: 645.6200
	Coverage: 100.00%
	Time taken (secs): 0.0047 s
	Stop reason: patience_exceeded

4: 4th instance of generation strategy 1. Path: instances/gen1/instance4.txt
Patience exhausted, no improvement to the objective solution in 1 ite

## Configuração 2 - PADRÃO+ALPHA:
GRASP PADRÃO mas com parâmetro α2 .

In [ ]:
config_2 = {
    "construction_method": "traditional",
    "construction_args": [alpha_2],
    "local_search_method": "first_improve",
}
results_df = run_experiment(config_2)

with open("pickles/results_config_2.pkl", "wb") as f:
    pickle.dump(results_df, f)

1: 1th instance of generation strategy 1. Path: instances/gen1/instance1.txt
Patience exhausted, no improvement to the objective solution in 10 iterations, stopping GRASP.
	Best objective value: 4614.2500
	Elements selected: [78, 40, 114, 12, 13, 9, 11, 1, 15, 5, 8, 14, 25, 32, 33, 37, 38, 45, 10, 46, 27, 47, 48, 64, 68, 69, 71, 50, 72, 36, 65, 73, 28, 76, 80, 81, 86, 79, 87, 90, 91, 92, 41, 89, 94, 97, 98, 105, 110, 111, 61, 116, 54, 113, 118, 117, 51, 123, 23, 129, 131, 124, 136, 102, 133, 67, 142, 144, 145, 146, 115, 150, 151, 152, 154, 157, 158, 159, 161, 163, 167, 172, 169, 175, 178, 160, 179, 187, 174, 189, 112, 192, 195, 196, 126, 198, 171, 22, 60, 99, 109, 147, 148, 153, 182, 43, 128, 184, 197, 166, 173, 134, 85, 137, 139, 83, 44, 53, 135, 125, 93]
	Coverage: 100.00%
	Time taken (secs): 3.5244 s

2: 2th instance of generation strategy 1. Path: instances/gen1/instance2.txt
Patience exhausted, no improvement to the objective solution in 10 iterations, stopping GRASP.
	Best object

KeyboardInterrupt: 

## Configuração 3 - PADRÃO+BEST:
GRASP PADRÃO mas com best-improving.

In [ ]:
config_3 = {
    "construction_method": "traditional",
    "construction_args": [alpha_1],
    "local_search_method": "best_improve",
}
results_df = run_experiment(config_3)

with open("pickles/results_config_3.pkl", "wb") as f:
    pickle.dump(results_df, f)

## Configuração 4 - PADRÃO+HC1:
GRASP PADRÃO mas com método de construção alternativo 1.

In [ ]:
p_1 = 0.5
config_4 = {
    "construction_method": "random_plus_greedy",
    "construction_args": [alpha_1, p_1],
    "local_search_method": "first_improve",
}
results_df = run_experiment(config_4)

with open("pickles/results_config_4.pkl", "wb") as f:
    pickle.dump(results_df, f)

## Configuração 5 - PADRÃO+HC2:
GRASP PADRÃO mas com método de construção alternativo 2.

In [ ]:
p_2 = 0.1
config_5 = {
    "construction_method": "sampled_greedy",
    "construction_args": [alpha_1, p_1],
    "local_search_method": "first_improve",
}
results_df = run_experiment(config_5)

with open("pickles/results_config_5.pkl", "wb") as f:
    pickle.dump(results_df, f)